In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
%matplotlib widget

import poses.utils.lie_groups.SE3 as SE3
import poses.utils.lie_groups.se3 as se3

Right-hand convention

In [2]:
def interpolate_pose(pose1: np.ndarray, pose2: np.ndarray, t: float) -> np.ndarray:
    assert pose1.shape == (4, 4)
    assert pose2.shape == (4, 4)

    return pose1 @ se3.exp(t * SE3.log(SE3.inverse(pose1) @ pose2))

In [3]:
def interpolate_covariances(cov1: np.ndarray, cov2: np.ndarray, t: float) -> np.ndarray:
    assert type(cov1) is type(cov2)
    assert t >= 0.0 and t <= 1.0

    if isinstance(cov1, np.ndarray):
        return __exp_covariance((1 - t) * __log_covariance(cov1) + t * __log_covariance(cov2))
    elif isinstance(cov1, float):
        return (1 - t) * cov1 + t * cov2
    else:
        print("Covariance can only be float or numpy array")
        raise


def __log_covariance(cov: np.ndarray) -> np.ndarray:
    w, v = np.linalg.eig(cov)
    return v @ np.diag(np.log(w)) @ v.T


def __exp_covariance(algebra: np.ndarray) -> np.ndarray:
    w, v = np.linalg.eig(algebra)
    return v @ np.diag(np.exp(w)) @ v.T

In [4]:
start = SE3.inverse(np.eye(4)); print(start)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [5]:
end = SE3.inverse(np.array([
    [0.0, -1.0, 0.0, 1.0],
    [1.0, 0.0, 0.0, 1.0],
    [0.0, 0.0, 1.0, 0.0],
    [0.0, 0.0, 0.0, 1.0]
])); print(end)

[[ 0.  1.  0. -1.]
 [-1.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


In [6]:
start_cov = np.eye(6) * 0.1; print(start_cov)

[[0.1 0.  0.  0.  0.  0. ]
 [0.  0.1 0.  0.  0.  0. ]
 [0.  0.  0.1 0.  0.  0. ]
 [0.  0.  0.  0.1 0.  0. ]
 [0.  0.  0.  0.  0.1 0. ]
 [0.  0.  0.  0.  0.  0.1]]


In [7]:
end_cov = np.eye(6) * 0.4
print(end_cov)

[[0.4 0.  0.  0.  0.  0. ]
 [0.  0.4 0.  0.  0.  0. ]
 [0.  0.  0.4 0.  0.  0. ]
 [0.  0.  0.  0.4 0.  0. ]
 [0.  0.  0.  0.  0.4 0. ]
 [0.  0.  0.  0.  0.  0.4]]


In [8]:
def plot_pose(ax, trajectory, orientation):
    ax.scatter(trajectory[0],
               trajectory[1],
               trajectory[2])
    ax.quiver(trajectory[0], trajectory[1], trajectory[2], 
              orientation[0], orientation[1], orientation[2], 
              length=0.3)

In [9]:
def plot_covariance(ax, translation_cov, position):
    coefs = 1 / translation_cov  # Coefficients in a0/c x**2 + a1/c y**2 + a2/c z**2 = 1 
    # Radii corresponding to the coefficients:
    rx, ry, rz = 1/(np.pi * np.sqrt(coefs))

    # Set of all spherical angles:
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)

    # Cartesian coordinates that correspond to the spherical angles:
    # (this is the equation of an ellipsoid):
    x = position[0] + rx * np.outer(np.cos(u), np.sin(v))
    y = position[1] + ry * np.outer(np.sin(u), np.sin(v))
    z = position[2] + rz * np.outer(np.ones_like(u), np.cos(v))

    # Plot:
    ax.plot_surface(x, y, z,  rstride=4, cstride=4, color='c', alpha=0.35)

In [11]:
fig = plt.figure(figsize=plt.figaspect(1))
ax = fig.add_subplot(111, projection='3d')

@widgets.interact(t=(0.0, 1.0, 0.01))
def update(t = 0.0):
    interpolated_pose = interpolate_pose(start, end, t)
    interpolated_cov = interpolate_covariances(start_cov, end_cov, t)
    
    print("Interpolated pose")
    print(interpolated_pose)
    print("\nInterpolated covariance")
    print(interpolated_cov)
    
    trajectory = SE3.inverse(interpolated_pose)[0:3, 3]
    orientation = SE3.inverse(interpolated_pose)[0:3, 1]
    translation_covariance = np.array([interpolated_cov[0,0], interpolated_cov[1,1], interpolated_cov[2,2]])
    
    plt.cla()
    ax.set_xlim3d(0, 1)
    ax.set_ylim3d(0, 1)
    ax.set_zlim3d(0, 1)
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    
    plot_pose(ax, trajectory, orientation)
    plot_covariance(ax, translation_covariance, trajectory)
    
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…